In [1]:
proxies = {'https':'85.25.95.117:9999',
           'https':'190.145.200.126:9999' ,
           'https':'20.47.108.204:9999' ,
           'https':'54.36.176.76:9999' ,
           'https':'167.114.174.168:9999' ,
           'https':'24.14.225.100:9999' ,
           'https':'212.129.15.88:9999' ,
           'https':'5.189.184.6:9999' ,
          }

In [2]:
import random

In [3]:
import requests
from bs4 import BeautifulSoup
import csv
import time

# open csv
fd = open("samples.csv", "w", encoding='utf-8')
writer = csv.writer(fd)
writer.writerow(["movie", "sentence", "score"])

print("크롤링 시작..")
for idx in range(1, 150000):
    if idx % 100  == 0:
            print(f"크롤링 중.. iter={idx}")

    # DDOS 공격 의심을 피하기 위한 sleep
    time.sleep(1)


    uri = f"https://movie.naver.com/movie/point/af/list.naver?&page={idx}"
    i = random.choice(list(proxies.values()))
    gain = requests.get(uri, i)
    html = BeautifulSoup(gain.content, 'html.parser')

    # 유의미한 내용이 담긴 content까지 접근
    dep0 = html.find("tbody")
    dep1 = dep0.findAll("tr")
    for i in range(len(dep1)):
        dep2 = dep1[i].find("td", {"class": "title"})
        dep3 = dep2.text

        # parsing
        content = dep3.split('\n')
        movie_review = content[5]
        if not movie_review: continue

        movie_title = content[1]
        movie_score = int(content[3].split('중')[-1])

        # to csv
        writer.writerow([movie_title, movie_review, movie_score])

print("완료!")
fd.close()

크롤링 시작..
크롤링 중.. iter=100
크롤링 중.. iter=200
크롤링 중.. iter=300
크롤링 중.. iter=400
크롤링 중.. iter=500
크롤링 중.. iter=600
크롤링 중.. iter=700
크롤링 중.. iter=800
크롤링 중.. iter=900
크롤링 중.. iter=1000


AttributeError: ignored

In [4]:
import csv
from collections import namedtuple


class RawMoviewReview(object):
    def __init__(self, file_name: str = "samples.csv"):
        self._file_name = file_name
        self._dataset = self.disk2program(file_name)

    def __len__(self):
        return len(self._dataset)

    def __getitem__(self, idx):
        return self._dataset[idx]

    def disk2program(self, file_name):
        MovieInfo = namedtuple("MovieInfo", ['title', 'comment', 'score'])
        with open(file_name, "r") as fd:
            reader = csv.reader(fd)

            return_data = []
            next(reader)  # skip first row
            for row_info in reader:
                title, comment, score = row_info
                info_obj = MovieInfo(title, comment, int(score))
                return_data.append(info_obj)
        return return_data


if __name__ == "__main__":
    dataset = RawMoviewReview()
    print(f"dataset[0] = {dataset[0]}")
    print(f"len(dataset) = {len(dataset)}")

dataset[0] = MovieInfo(title='마미', comment='이게 평점이 맞냐?  이건  예술영화도,  가족영화도아니고  . 대화도, 욕도  아무것도할줄모르는 멍청한감독이 만든 쓰레기영화  아트나인에서 시간이 제발흐르길 바라며봣다.  너무답답해서  십오분 남기고뛰쳐나옴  쓸대없이 런닝타임 겁나기네  아  쌍욕나와 ', score=1)
len(dataset) = 9312


In [5]:
class MoviewReview(RawMoviewReview):
    def __init__(self, score_threshold: int, file_name: str = "samples.csv"):
        super().__init__(file_name)
        self._score_threshold = score_threshold

    def __getitem__(self, idx):
        positive_or_negative = self._dataset[idx].score >= self._score_threshold
        return self._dataset[idx].comment.strip(), positive_or_negative


if __name__ == "__main__":
    dataset = MoviewReview(score_threshold=5)
    print(dataset[2])

('결국엔 미지에서 미궁 속으로', True)
